In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
import pickle
from sklearn.inspection import permutation_importance

**LOADING DATA**

In [3]:
# Data with missing values (for XGB only)
df_missing = pd.read_csv('../Preprocessed Data/df_regression_not_imp.csv')

# Data without missing values 
df_imp = pd.read_csv('../Preprocessed Data/df_regression_imp.csv')

**CREATING FUNCTION**

This function
1. Splits the preprocessed data
2. Runs the model
3. Saves the best model and parameters as a pkl file

In [3]:
def MLpipe_KFold(df, target, ML_algo, param_grid):
    '''
    This function splits the data to other/test (80/20) and then applies KFold with 4 folds to other.
    The accuracy is minimized in cross-validation.

    1. Loop through  different 5 random states
    2. Split data 
    3. Fit a model using GridSearchCV with KFold
    4. Calculate the model's error on the test set 
    5. Return a list of 5 test scores and 5 best models 

    @params:
        df: preprocessed pandas dataframe
        target: string with column name of target variable
        ML_also: specific algorithm to train and test
        param_grid: dictionary with parameters for each model
    '''

    y = df[target]
    X = df.drop(columns=[target])
    
    # Results dictionary to be returned
    results = {}
    baseline_rmse = []

    for i in range(5):

        # Split Data
        X_other, X_test, y_other, y_test = train_test_split(X,y, train_size=0.8,random_state = 42*i)

        y_baseline_pred = np.full_like(y_test, np.mean(y_other))  # Predict the mean of training set for baseline
        baseline_rmse_value = np.sqrt(mean_squared_error(y_test, y_baseline_pred))
        baseline_rmse.append(baseline_rmse_value)

        # K-folds
        kf = KFold(n_splits=4,shuffle=True,random_state=42*i)

        # Make custom RMSE
        def rmse(y_true, y_pred):
            return np.sqrt(mean_squared_error(y_true, y_pred))
        
        rmse_scorer = make_scorer(rmse, greater_is_better=False)

        # CV
        grid = GridSearchCV(ML_algo, param_grid=param_grid,scoring = rmse_scorer,
                                cv=kf, return_train_score = True, n_jobs=-1, verbose=False)

        grid.fit(X_other, y_other)
        y_pred = grid.best_estimator_.predict(X_test)

        results[i] = {
            'X_test': X_test,
            'y_test': y_test,
            'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
            'best_model': grid.best_estimator_,
            'best_params': grid.best_params_
        }

    results['baseline_rmse'] = {
        'baseline_avg': np.mean(baseline_rmse),
        'baseline_std': np.std(baseline_rmse)
    }

    baseline = results['baseline_rmse']['baseline_avg']

    for i in range(5):
        results[i]['relative_improvement'] = (baseline - results[i]['test_rmse'])/baseline


    return results

**XGBOOST**

In [4]:
params_XGB = {
    'learning_rate': [0.03],
    'n_estimators': [500],  # Number of trees
    'max_depth': [1, 3, 10, 30, 100],  # Depth of the tree
    'subsample': [0.33, 0.5, 0.66],  # Fraction of samples used for fitting trees
    'colsample_bytree': [0.5, 0.75, 1.0],  # Fraction of features used for fitting trees
    'scale_pos_weight': [0.5, 1, 2, 5, 10, 15]
}

ML_algo = XGBRegressor(random_state=42)
params = params_XGB

XGBresults = MLpipe_KFold(df_missing, target='target', ML_algo=ML_algo, param_grid=params)


/users/rparik14/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/users/rparik14/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/users/rparik14/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/users/rparik14/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_tr

XGBoostError: [15:54:40] /home/conda/feedstock_root/build_artifacts/xgboost-split_1727635009624/work/src/data/data.cc:514: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) /users/rparik14/anaconda3/envs/data1030/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x6e) [0x7fb474ee860e]
  [bt] (1) /users/rparik14/anaconda3/envs/data1030/lib/libxgboost.so(xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const&, xgboost::StringView, xgboost::Json)+0xbb1) [0x7fb475178c91]
  [bt] (2) /users/rparik14/anaconda3/envs/data1030/lib/libxgboost.so(xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView)+0x375) [0x7fb47517a985]
  [bt] (3) /users/rparik14/anaconda3/envs/data1030/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0x7f) [0x7fb474ddf31f]
  [bt] (4) /users/rparik14/anaconda3/envs/data1030/lib/python3.12/lib-dynload/../../libffi.so.8(+0x6a4a) [0x7fb5cedcaa4a]
  [bt] (5) /users/rparik14/anaconda3/e

In [ ]:
for i in range(5):
    print(results[i]['test_rmse'])
    print(results[i]['best_params'])

15.22948940352569
{'C': 1000.0}
15.092411489642041
{'C': 1000.0}
16.407819339750255
{'C': 1000.0}
14.711758407542776
{'C': 1000.0}
7.950504511262258
{'C': 1000.0}


**OTHER REGRESSION MODELS**

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

max_iter=10000
random_state=42

models_and_params = {
    'Linear': {'model': LinearRegression(),
               'params': {'fit_intercept': [True]}    
    },

    'Ridge': {'model': Ridge(random_state=random_state, max_iter=max_iter),
              'params': {'alpha': np.logspace(-8, 3, 12),
                         'fit_intercept': [True]}
    },

    'Lasso': {'model': Ridge(random_state=random_state, max_iter=max_iter),
              'params': {'alpha': np.logspace(-8, 3, 12),
                         'fit_intercept': [True]}
    },
    
    'SVR Linear': {'model': SVR(kernel = 'linear'),
                   'params': {'C': np.logspace(-5, 3, 9),}
    },
    'SVR RBF': {'model': SVR(kernel = 'rbf'),
                'params': {'C': np.logspace(-5, 3, 9)}
    },

    'SVR Poly': {'model': SVR(kernel = 'poly'),
                'params': {'C': np.logspace(-5, 3, 9),
                            'degree': [2,3,4,5]
                }
    }
}

In [ ]:
'''
Performing grid search.
'''
models = ['Linear', 'Ridge', 'Lasso', 'SVR Linear', 'SVR RBF', 'SVR Poly']

model_results = {}
model_results['XGB'] = XGBresults

for model in models:
    ML_algo = models_and_params[model]['model']
    params = models_and_params[model]['params']
    
    print(f"Results for {model}")
    results = MLpipe_KFold(df=df_imp, target='target', ML_algo=ML_algo, param_grid=params)

    # To print for each random_state
    rmses = []

    for i in range(5):
        print(f"Results for Random State {i}")
        print(f"  Test RMSE: {results[i]['test_rmse']}")
        print(f"  Baseline: {results['baseline_rmse']['baseline_avg']}, Relative RMSE Improvement: {results[i]['relative_improvement']}")
        print(f"  Baseline Standard Deviation: {results['baseline_rmse']['baseline_std']}")
        print(f"  Best Params: {results[i]['best_params']}")
        rmses.append(results[i]['test_rmse'])

    print(f"Mean of Test RMSEs: {np.mean(rmses)}")
    print(f"Standard Deviation of Test RMSEs: {np.std(rmses)}")
    print("=========")

    model_results[model] = results

# Save entire dictionary as pickle
import pickle 

with open('model_results.pkl', 'wb') as f:
    pickle.dump(model_results, f)

Results for Linear
Results for Random State 0
  Test RMSE: 13.640918934213818
  Baseline: 21.756619533559224, Relative RMSE Improvement: 0.37302213180807214
  Baseline Standard Deviation: 3.562615117010165
  Best Params: {'fit_intercept': True}
Results for Random State 1
  Test RMSE: 3.997559493408203
  Baseline: 21.756619533559224, Relative RMSE Improvement: 0.8162600817998387
  Baseline Standard Deviation: 3.562615117010165
  Best Params: {'fit_intercept': True}
Results for Random State 2
  Test RMSE: 3.0596038490161854
  Baseline: 21.756619533559224, Relative RMSE Improvement: 0.8593713584825622
  Baseline Standard Deviation: 3.562615117010165
  Best Params: {'fit_intercept': True}
Results for Random State 3
  Test RMSE: 6.833289216482677
  Baseline: 21.756619533559224, Relative RMSE Improvement: 0.685921371840766
  Baseline Standard Deviation: 3.562615117010165
  Best Params: {'fit_intercept': True}
Results for Random State 4
  Test RMSE: 7.693105968229396
  Baseline: 21.7566195335

**GETTING BEST MODELS**

In [ ]:
for model in model_results.keys():
    improvements = []
    for i in range(5):
        improvements.append(model_results[model][i]['relative_improvement'])
    
    print(model)
    print(np.argmax(improvements))
    print(np.max(improvements))

XGB
2
0.3076779815867441
Linear
2
0.8593713584825622
Ridge
2
0.8496201533641647
Lasso
2
0.8496201533641647
SVR Linear
1
0.8933834180596489
SVR RBF
2
0.6159578778759225
SVR Poly
4
0.6345707797574556


**FEATURE IMPORTANCE**

In [ ]:
best_svr = model_results['SVR Linear'][1]['best_model']

In [ ]:
feature_importance = pd.DataFrame({
    'Feature': df_imp.drop(columns='target').columns, 
    'Coefficient': best_svr.coef_[0]
})

feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False).drop(columns='Abs_Coefficient')

feature_importance

,Feature,Coefficient
5,std__birth-1yr_wt_diff,-2.537441e+01
2,std__3yr_wt_pct,9.065543e+00
1,std__1yr_wt_pct,8.412943e+00
6,std__1yr-3yr_wt_diff,-6.951062e+00
21,ohot__6mo_feeding_type_Both Breast and Formula,1.695883e+00
9,ohot__race_Other,-1.437654e+00
18,ohot__4mo_feeding_type_Breast Feeding,-5.177217e-01
8,ohot__race_Hispanic,3.700030e-01
24,ohot__6mo_feeding_type_missing,-3.670397e-01
7,ohot__race_Black,3.309181e-01


In [ ]:
from sklearn.inspection import permutation_importance

X_test = model_results['SVR Linear'][1]['X_test']
y_test = model_results['SVR Linear'][1]['y_test']

# Calculate permutation importance
perm_importance = permutation_importance(best_svr, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

# Create a DataFrame to store feature names and their importance scores
perm_importance_df = pd.DataFrame({
    'Feature': df_imp.drop(columns='target').columns,
    'Permutation Importance': perm_importance.importances_mean
})

# Sort by absolute permutation importance (in descending order)
perm_importance_df = perm_importance_df.sort_values(by='Permutation Importance', ascending=False)

# Display the sorted permutation importance
perm_importance_df

,Feature,Permutation Importance
5,std__birth-1yr_wt_diff,2.880027
1,std__1yr_wt_pct,0.165099
6,std__1yr-3yr_wt_diff,0.087539
2,std__3yr_wt_pct,0.070058
9,ohot__race_Other,0.006838
24,ohot__6mo_feeding_type_missing,0.001817
19,ohot__4mo_feeding_type_Formula Feeding,0.000582
11,ohot__race_missing,0.000413
4,std__age_on_obes,0.000267
10,ohot__race_White,0.000219
